In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import calendar

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [2]:
# %timeit sirve para evaluar el tiempo de ejecucion
event = pd.read_csv('events_up_to_01062018.csv')
label_train = pd.read_csv('labels_training_set.csv')
test = pd.read_csv('trocafone_kaggle_test.csv')

#projects_2017.head()trocafone_kaggle_test
#projects_2017

In [3]:
test_con_feature  = pd.read_csv('test_paso_5.csv')
train_con_feature = pd.read_csv('train_paso_5.csv')

In [4]:
events_test = event.merge(test,on='person',how='inner')
events_test.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,"9,288.00",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:30:30,viewed product,4886f805,NaN,"9,288.00",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:07:23,search engine hit,4886f805,NaN,nan,NaN,NaN,NaN,NaN,NaN,...,Google,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:56,checkout,4886f805,NaN,"9,288.00",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:35,viewed product,4886f805,NaN,"9,287.00",Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#feature engine trucho
events_training = event.merge(label_train,on='person',how='inner')

In [11]:
 cualidades = {"Moto":["Moto"],
   "j7":["j7"],
  "s8":["s8"],
    "Lg":["Lg"],
          "A5":["A5"],
          "Celular":["Celular"],
               "Motorola":["Motorola"],
               "Galaxy":["Galaxy"],
               "Samsung":["Samsung"],
               "Iphone":["Iphone"],
               "prime":["prime"],
                "sansung":["sansung"],
               "s6":["s6"],
               "j5":["j5"],
               "j2":["j2"],
               "edge":["edge"],
               "dourado":["dourado"],
          "g5":["g5"]        

  }

In [12]:
def agregarcolumnas(archivo,archivo_con_feature):
    #archivo_con_feature es un archivo test o train con feature antes calculada
    #archivo es el train con label o el test
    
     
    archivo['timestamp'] = pd.to_datetime(archivo['timestamp'])
    archivo['Anio'] = archivo['timestamp'].map(lambda x:x.year)
    archivo['Mes'] = archivo['timestamp'].map(lambda x:x.month)
    archivo['Dia'] = archivo['timestamp'].map(lambda x:x.dayofweek)
    archivo['Mes_Nombre'] = archivo['timestamp'].map(lambda x:x.month)

    archivo['Hora'] = pd.to_datetime(archivo['timestamp'],\
                                   format='%H:%M',errors='coerce').dt.hour
    
    
    for cualidad in cualidades:
               
                archivo[cualidad]= archivo['search_term'].str.lower().str.contains(cualidad)
                  
            
    for cualidad in cualidades:
        archivo[cualidad].fillna(0, inplace=True)           
    
    
    
    final_letras=archivo.groupby('person').agg({'Moto':'sum' ,'j7':'sum','s8':'sum','Lg':'sum' ,'A5':'sum','Celular':'sum' ,'Motorola':'sum','Galaxy':'sum','Samsung':'sum','Iphone':'sum','prime':'sum','sansung':'sum','s6':'sum','j5':'sum','edge':'sum','j2':'sum','dourado':'sum','g5':'sum'})
    
    final_letras = final_letras.rename(columns={'Moto':'Moto_term' ,'j7':'j7_term','s8':'s8_term','Lg':'lg_term' ,'A5':'A5_term','Celular':'celular_term' ,'Motorola':'motorola_term','Galaxy':'galaxy_term','Samsung':'samsung_term','Iphone':'iphone_term','prime':'prime_term','sansung':'sansung_term','s6':'s6_term','j5':'j5_term','edge':'edge_term','j2':'j2_term','dourado':'dourado_term','g5':'g5_term'}) 
    
    archivo_listapalabras=pd.merge(archivo_con_feature,final_letras,on='person',how='left')
    archivo_listapalabras=archivo_listapalabras.fillna(0)
    
    
     
  
 
    
    
    return archivo_listapalabras

In [13]:
data_train=agregarcolumnas(events_training,train_con_feature)
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19414 entries, 0 to 19413
Columns: 104 entries, person to g5_term
dtypes: float64(72), int64(31), object(1)
memory usage: 15.6+ MB


In [ ]:
len(data_train.columns)

In [14]:
data_test=agregarcolumnas(events_test,test_con_feature)
data_test.head() 

,person,mean_time_between_events,mean_time_between_co,mean_time_between_ch,mean_time_between_vs,mean_time_between_ad,mean_time_between_bl,mean_time_between_st,mean_time_between_se,mean_time_between_sp,...,samsung_term,iphone_term,prime_term,sansung_term,s6_term,j5_term,edge_term,j2_term,dourado_term,g5_term
0,4886f805,2.24,-1.00,0.00,0.00,-1.00,-1.00,-1.00,0.00,0.00,...,0,0,0,0,0,0,0,0,0,0
1,0297fc1e,4.33,-1.00,6.25,5.10,5.60,6.34,-1.00,-1.00,6.18,...,0,0,0,0,0,0,0,0,0,0
2,2d681dd8,4.53,-1.00,0.00,5.93,0.00,5.33,-1.00,5.93,0.00,...,0,0,0,0,0,0,0,0,0,0
3,cccea85e,3.39,-1.00,0.00,4.99,5.16,5.52,4.39,4.91,0.00,...,0,0,0,0,0,0,0,0,0,0
4,4c8a8b93,3.13,-1.00,4.70,4.26,4.30,2.74,-1.00,4.25,2.83,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
len(data_test.columns)

In [15]:
data_train.to_csv("train_paso_8.csv", index = False)

In [16]:
data_test.to_csv("test_paso_8.csv", index = False)